In [1]:
import os
import pickle
import langchain
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

In [2]:
from dotenv import load_dotenv

load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [3]:
llm = GoogleGenerativeAI(model="gemini-1.0-pro", temperature=0.9, max_tokens=500)

In [4]:
loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load()
len(data)

2

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
docs = text_splitter.split_documents(data)

In [6]:
len(docs)

16

In [7]:
docs[2]

Document(metadata={'source': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html'}, page_content='Story continues below Advertisement\n\nRemove Ad\n\nOther megacaps also rose, with AmazonÂ (AMZN.O)Â climbing 3.5% and MicrosoftÂ (MSFT.O)Â adding 1.1%.\n\nMeta PlatformsÂ (META.O)Â jumped 3.25% after a report on Sunday said the social media platform was working on a new, more powerful AI system.\n\nWalt DisneyÂ (DIS.N)Â added 1.2% and Charter CommunicationsÂ (CHTR.O)Â rose 3.2% after theyÂ reached a dealÂ for Disney\'s programming, including ESPN, to return to the Spectrum cable service just hours ahead of the start of NFL "Monday Night Football."\n\nRelated stories\n\nS&P 500 ends up, but near flat for week after Monday\'s steep selloff\n\nS&P 500 and Nasdaq rebound as Fed eases recession fears\n\nNasdaq, S&P 500 fall 3% each amid US recession fears, Apple drop\n\nInvestors are looking to August consumer price index data due o

In [8]:
embeddings = OllamaEmbeddings()

vector_index_genai = FAISS.from_documents(docs, embeddings)

In [9]:
file_path = "vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vector_index_genai, f)

In [10]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

In [11]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

RetrievalQAWithSourcesChain(combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=GoogleGenerativeAI(model='gemini-1.0-pro', temperature=0.9, client=genai.GenerativeModel(
    model_name='models/gemini-1.0-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
))), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Do

In [12]:
query = "what is the price of Tiago iCNG?"

langchain.debug = True
chain({"question": query}, return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what is the price of Tiago iCNG?"
}


C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Set Alert\n\nlive\n\nbselive\n\nnselive\n\nVolume\n\nTodays L/H\n\nMore\n\nTata Motors on Friday launched the CNG variant of its micro SUV Punch priced between Rs 7.1 lakh and Rs 9.68 lakh (ex-showroom, Delhi).\n\nThe Punch iCNG is equipped with the company's proprietary twin-cylinder technology with enhanced safety features like a micro-switch to keep the car switched off at the time of refuelling and thermal incident protection that cuts off CNG supply to the engine and releases gas into the atmosphere, Tata Motors said in a statement.\n\nStory continues below Advertisement\n\nRemove Ad\n\nIt is also equipped with other features such as voice assisted electric sunroof, automatic projector head

{'answer': 'The text does not mention the price of Tiago iCNG.\n',
 'sources': ''}